In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import pandas as pd
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist', validation_size=0, one_hot=True)
test_len = mnist.test.images.shape[0]
validation_len = int(test_len * 0.1)
num_label=10
train_x = mnist.train.images
test_x = mnist.test.images[validation_len : test_len, :]
validation_x = mnist.test.images[ : validation_len, :]

train_y = mnist.train.labels
test_y = mnist.test.labels[validation_len : test_len]
validation_y = mnist.test.labels[ : validation_len]

print("\nTraining X shape: " + str(train_x.shape))
print("Testing X shape: " + str(test_x.shape))
print("Validation X shape: " + str(validation_x.shape))

print("\nTraining Y shape: " + str(train_y.shape))
print("Testing Y shape: " + str(test_y.shape))
print("Validation Y shape: " + str(validation_y.shape))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [4]:
import scipy.io as sio
import scipy.misc as scimisc
safe_list = [0,2,5,6,8,12,13,14,15,16,17,18,19,21,26]  # other alphabets have characters which look like digits
m = sio.loadmat("./data/data_background.mat")

squished_set = []
for safe_number in safe_list:
    for alphabet in m['images'][safe_number]:
        for letters in alphabet:
            for letter in letters:
                for example in letter:
                    squished_set.append(scimisc.imresize(1 - example[0], (28,28)).reshape(1, 28*28))

safe_images = np.concatenate(squished_set, axis=0)

/Users/senqicao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


In [36]:
M = 100
D=7*7*64
h1=512
D2=10

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

tf.reset_default_graph()
# variables
X = tf.placeholder(tf.float32, shape = [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None,10])

w_conv1 = tf.get_variable('w_conv1', [5,5, 1,32], initializer = tf.contrib.layers.xavier_initializer())
w_conv2 = tf.get_variable('w_conv2', [5,5,32,64], initializer = tf.contrib.layers.xavier_initializer())

w0 = tf.get_variable('w_fc1', [7*7*64, h1], initializer = tf.contrib.layers.xavier_initializer())
b0 = tf.get_variable('b_fc1', [1,h1], initializer = tf.contrib.layers.xavier_initializer())
w1 = tf.get_variable('w_fc2', [512, 10], initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('b_fc2', [1,10], initializer = tf.contrib.layers.xavier_initializer())

# network
b=True
con1 = tf.nn.conv2d(X, w_conv1, strides=[1, 1, 1, 1], padding='SAME')
h_conv1 = tf.nn.relu(tf.layers.batch_normalization(con1, training=b))
h_pool1 = max_pool_2x2(h_conv1)

con2 = tf.nn.conv2d(h_pool1, w_conv2, strides=[1, 1, 1, 1], padding='SAME')
h_conv2 = tf.nn.relu(tf.layers.batch_normalization(con2, training=b))
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.layers.flatten(h_pool2)

h = tf.nn.relu(tf.matmul(h_pool2_flat, w0) + b0)
logits = tf.matmul(h, w1) + b1
output = tf.nn.softmax(logits)#batch*10

#evaluation
probs = tf.reduce_sum(output*Y,1)#batch*1
p=tf.reduce_mean(probs,0)#积分
#cross_ent = tf.reduce_mean(tf.reduce_sum(output*tf.log(output+1e-11),1),0)
cross_ent = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=Y)
max_p = tf.reduce_max(output,1)
ent = tf.reduce_sum(-tf.log(output+1e-11)*output,1)


correct_pred = tf.equal(tf.argmax(output,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# training
a = 0.00001
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(a).minimize(cross_ent)
saver = tf.train.Saver(max_to_keep = 200)

with tf.Session() as sess:
    start_time = time.time()
    #sess.run(tf.global_variables_initializer())
    model_id=0
    save_path = saver.restore(sess, "./baseline_point_estimation/classification_mnist60/model.ckpt")
    for i in range(90000):
        batch = mnist.train.next_batch(M)
        x_batch = np.reshape(batch[0],[M, 28, 28, 1])
        y_batch = batch[1]
        
        if i%100==0:
            b=False
            maxp_OOD = []
            ent_OOD = []
            
            test_accuracy = []
            maxp_in = []
            ent_in = []
            #accu = sess.run(accuracy,{X: np.reshape(mnist.train.images,[55000,28,28,1]), Y: mnist.train.labels})
                        
            
            for j in range(0,100):
                imgs_in = mnist.test.images[j*100:(j+1)*100]
                labels_in = mnist.test.labels[j*100:(j+1)*100]
                acc = sess.run(accuracy,{X:np.reshape(imgs_in,[100,28,28,1]),Y:labels_in})
                maxp_in1 = sess.run(max_p,{X:np.reshape(imgs_in,[100,28,28,1])})
                ent_in1 = sess.run(ent,{X:np.reshape(imgs_in,[100,28,28,1])})
                test_accuracy.append(acc)
                maxp_in.extend(maxp_in1)
                ent_in.extend(ent_in1)
            test_accuracy = np.array(test_accuracy)
            maxp_in = np.array(maxp_in)
            ent_in = np.array(ent_in)
                
            for k in range(0,92):
                imgs_OOD = safe_images[k*100:(k+1)*100]
                maxp_OOD1 = sess.run(max_p,{X:np.reshape(imgs_OOD,[100,28,28,1])})
                ent_OOD1 = sess.run(ent,{X:np.reshape(imgs_OOD,[100,28,28,1])})
                maxp_OOD.extend(maxp_OOD1)
                ent_OOD.extend(ent_OOD1)
            maxp_OOD = np.array(maxp_OOD)
            ent_OOD = np.array(ent_OOD)

            validation_error = sess.run(accuracy, {X:np.reshape(validation_x,[1000,28,28,1]), Y:validation_y})
            test_error = np.mean(np.array(test_accuracy))
            '''
            test_error = sess.run(accuracy, {X:np.reshape(mnist.test.images,[10000,28,28,1]), Y:mnist.test.labels[:5000]})
            validation_error = sess.run(accuracy, {X:np.reshape(validation_x,[1000,28,28,1]), Y:validation_y})
            maxp_OOD = sess.run(max_p,{X:np.reshape(safe_images,[9200,28,28,1])})
            maxp_in = sess.run(max_p, {X:np.reshape(mnist.test.images,[10000,28,28,1])})
            ent_OOD = sess.run(ent,{X:np.reshape(safe_images,[9200,28,28,1])})
            ent_in = sess.run(ent, {X:np.reshape(mnist.test.images,[10000,28,28,1])})
            '''
            print ("time:",i, time.time() - start_time, "test accuracy", test_error, "validation_error",validation_error)
            
            print("maxp_OOD:",np.mean(maxp_OOD>0.99),np.mean(maxp_OOD),np.std(maxp_OOD))
            print("maxp_inD:",np.mean(maxp_in>0.99),np.mean(maxp_in),np.std(maxp_in))
            print("ent_OOD:",np.mean(ent_OOD),np.std(ent_OOD), "ent_in:", np.mean(ent_in),np.std(ent_in))
            
            safe, risky  = -np.reshape(maxp_in,[10000,1]), -np.reshape(maxp_OOD,[9200,1])
            labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
            #labels[:safe.shape[0]] += 1
            labels[safe.shape[0]:] += 1
            examples = np.squeeze(np.vstack((safe, risky)))
            print('AUPR_p:', round(100*average_precision_score(labels, examples), 2))
            print('AUROC_p:', round(100*roc_auc_score(labels, examples), 2))

            safe, risky = np.reshape(ent_in,[10000,1]), np.reshape(ent_OOD,[9200,1])
            labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
            #labels[:safe.shape[0]] += 1
            labels[safe.shape[0]:] += 1
            examples = np.squeeze(np.vstack((safe, risky)))
            print('AUPR_entropy:', round(100*average_precision_score(labels, examples), 2))
            print('AUROC_entropy:', round(100*roc_auc_score(labels, examples), 2))
            print("############################################")
            
        if i < 1000:
            a = 0.005
            a = 0.00005
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 5000:
            a = 0.001
            a = 0.00005
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 20000:
            a = 0.0005
            a = 0.00005
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 30000:
            a = 0.0001
            a = 0.00001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        elif i < 40000:
            a = 0.00005
            a = 0.00001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        else:
            a = 0.00001
            a = 0.000001
            b = True
            sess.run(train_step,{X: x_batch, Y: y_batch})
        
        if i%500==0:
            #save_path = saver.save(sess, "./baseline_point_estimation/classification_mnist%s/model.ckpt" % model_id)
            model_id+=1

INFO:tensorflow:Restoring parameters from ./baseline_point_estimation/classification_mnist60/model.ckpt
time: 0 28.855525970458984 test accuracy 0.99050003 validation_error 0.99
maxp_OOD: 0.2614130434782609 0.8316217 0.18502985
maxp_inD: 0.9621 0.9952472 0.035239115
ent_OOD: 0.44568175 0.4122605 ent_in: 0.013703915 0.08283482
AUPR_p: 95.37
AUROC_p: 96.51
AUPR_entropy: 95.68
AUROC_entropy: 96.63
############################################
time: 100 81.15030717849731 test accuracy 0.9915 validation_error 0.991
maxp_OOD: 0.2539130434782609 0.82838684 0.18589777
maxp_inD: 0.9631 0.99504775 0.03716792
ent_OOD: 0.4538844 0.4139212 ent_in: 0.013863315 0.083727404
AUPR_p: 95.39
AUROC_p: 96.6
AUPR_entropy: 95.8
AUROC_entropy: 96.72
############################################
time: 200 134.13733625411987 test accuracy 0.99050003 validation_error 0.99
maxp_OOD: 0.2482608695652174 0.82742107 0.18558742
maxp_inD: 0.9626 0.99502325 0.036333937
ent_OOD: 0.45714068 0.4136153 ent_in: 0.014117738 0.08

KeyboardInterrupt: 

In [31]:
60000*50/100/500

60.0

In [37]:
tf.reset_default_graph()
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
h1=512
a = 0.0005
num_iter = 5000
batch_size = 100
networks = ['network1', 'network2', 'network3', 'network4', 'network5']
b=True
def get_network(network_name):
    x_image = tf.placeholder(tf.float32, shape = [None, 28, 28, 1])

    with tf.variable_scope(network_name):
        w_conv1 = tf.get_variable(network_name + 'w_conv1', [5,5, 1,32], initializer = tf.contrib.layers.xavier_initializer())
        w_conv2 = tf.get_variable(network_name + 'w_conv2', [5,5,32,64], initializer = tf.contrib.layers.xavier_initializer())
        w0 = tf.get_variable(network_name + 'w_fc1', [7*7*64, h1], initializer = tf.contrib.layers.xavier_initializer())
        b0 = tf.get_variable(network_name + 'b_fc1', [1,h1], initializer = tf.contrib.layers.xavier_initializer())
        w1 = tf.get_variable(network_name + 'w_fc2', [512, 10], initializer = tf.contrib.layers.xavier_initializer())
        b1 = tf.get_variable(network_name + 'b_fc2', [1,10], initializer = tf.contrib.layers.xavier_initializer())
    # network
    con1 = tf.nn.conv2d(x_image, w_conv1, strides=[1, 1, 1, 1], padding='SAME')
    h_conv1 = tf.nn.relu(tf.layers.batch_normalization(con1, training=b))
    h_pool1 = max_pool_2x2(h_conv1)

    con2 = tf.nn.conv2d(h_pool1, w_conv2, strides=[1, 1, 1, 1], padding='SAME')
    h_conv2 = tf.nn.relu(tf.layers.batch_normalization(con2, training=b))
    h_pool2 = max_pool_2x2(h_conv2)

    h_pool2_flat = tf.layers.flatten(h_pool2)

    h = tf.nn.relu(tf.matmul(h_pool2_flat, w0) + b0)
    logits = tf.matmul(h, w1) + b1
    output = tf.nn.softmax(logits)

    y_label = tf.placeholder(tf.float32, shape = [None, 10])

    loss = tf.reduce_mean(tf.div(tf.reduce_sum(tf.square(tf.subtract(output, y_label)), axis = 1), num_label), axis = 0) 
   
    # Get trainable variables
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, network_name) 
    
    #update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    #with tf.control_dependencies(update_ops):
    train_opt = tf.train.AdamOptimizer(a).minimize(loss, var_list = train_vars)
    
    return x_image, y_label, output, loss, train_opt, train_vars

In [38]:
x_list = []
y_list = []
output_list = []
loss_list = []
train_list = []
train_var_list = []

# Train each ensemble network
for i in range(len(networks)):
    x_image, y_label, output, loss, train_opt, train_vars = get_network(networks[i])

    x_list.append(x_image)
    y_list.append(y_label)
    output_list.append(output)
    loss_list.append(loss)
    train_list.append(train_opt)
    train_var_list.append(train_vars)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [40]:
# Create Session
saver = tf.train.Saver(max_to_keep = 200)
config = tf.ConfigProto()
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())
model_id=0
#save_path = saver.restore(sess, "./ensemble/classification_mnist60/model.ckpt")

In [43]:
test_size = 10000

num_iter = 60000
batch_size = 100

train_data_num = train_x.shape[0]
test_data_num  = test_x.shape[0]
img_size=28
safe_images = np.reshape(safe_images,[9200,28,28,1])

for iter in range(num_iter):
    #test
    if iter % 1 == 0:
        x_batch_test = np.reshape(mnist.test.images,[10000, 28, 28, 1])
        y_batch_test = mnist.test.labels
        
        outputs=[]
        b=False
        for i in range(len(networks)):
            prob = sess.run(output_list[i], {x_list[i]: x_batch_test, y_list[i]: y_batch_test})
            outputs.append(prob)
        
        outputs=np.array(outputs) 
        softmax = np.mean(outputs,0)
        '''
        maxp_in = np.max(softmax,1)
        ent_in = np.sum(-np.log(softmax+1e-11)*softmax,1)
        Eent_in = np.mean(np.sum(-np.log(outputs+1e-11)*outputs,2),0)
        MI_in = ent_in - Eent_in
        '''
        acc = np.mean(np.argmax(softmax,1) ==  np.argmax(y_batch_test,1))
        print( iter, 'Final Testing Accuracy: ', acc)
        '''
        outputs_OOD=[]
        for j in range(len(networks)):
            prob_OOD = sess.run(output_list[j], {x_list[j]: safe_images})
            outputs_OOD.append(prob_OOD)
            
        outputs_OOD=np.array(outputs_OOD) 
        softmax_OOD = np.mean(outputs_OOD,0)
        maxp_OOD = np.max(softmax_OOD,1)
        ent_OOD = np.sum(-np.log(softmax_OOD+1e-11)*softmax_OOD,1)
        Eent_OOD = np.mean(np.sum(-np.log(outputs_OOD+1e-11)*outputs_OOD,2),0)
        MI_OOD = ent_OOD - Eent_OOD
        
        print("maxp_OOD:",np.mean(maxp_OOD>0.99),np.mean(maxp_OOD),np.std(maxp_OOD))
        print("maxp_inD:",np.mean(maxp_in>0.99),np.mean(maxp_in),np.std(maxp_in))
        print("ent_OOD:",np.mean(ent_OOD),np.std(ent_OOD), "ent_in:", np.mean(ent_in),np.std(ent_in))

        safe, risky  = -np.reshape(maxp_in,[10000,1]), -np.reshape(maxp_OOD,[9200,1])
        labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
        labels[safe.shape[0]:] += 1
        examples = np.squeeze(np.vstack((safe, risky)))
        print('AUPR_p:', round(100*average_precision_score(labels, examples), 2))
        print('AUROC_p:', round(100*roc_auc_score(labels, examples), 2))

        safe, risky = np.reshape(ent_in,[10000,1]), np.reshape(ent_OOD,[9200,1])
        labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
        labels[safe.shape[0]:] += 1
        examples = np.squeeze(np.vstack((safe, risky)))
        print('AUPR_entropy:', round(100*average_precision_score(labels, examples), 2))
        print('AUROC_entropy:', round(100*roc_auc_score(labels, examples), 2))
        
        safe, risky = np.reshape(MI_in,[10000,1]), np.reshape(MI_OOD,[9200,1])
        labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
        labels[safe.shape[0]:] += 1
        examples = np.squeeze(np.vstack((safe, risky)))
        print('AUPR_MI:', round(100*average_precision_score(labels, examples), 2))
        print('AUROC_MI:', round(100*roc_auc_score(labels, examples), 2))
        print("############################################")
        '''
    #train
    batch = mnist.train.next_batch(batch_size)
    x_batch = np.reshape(batch[0],[batch_size, 28, 28, 1])
    y_batch = batch[1]
    if iter < 1000:
        a = 0.005
    elif iter < 5000:
        a = 0.001
    elif iter < 20000:
        a = 0.0005
    elif iter < 30000:
        a = 0.0001
    elif iter < 40000:
        a = 0.00005
    else:
        a = 0.00001
    for k in range(len(networks)):
        b = True
        sess.run(train_list[k], {x_list[k]:x_batch, y_list[k]: y_batch})

    

0 Final Testing Accuracy:  0.0971
1 Final Testing Accuracy:  0.1729
2 Final Testing Accuracy:  0.2686
3 Final Testing Accuracy:  0.1975


KeyboardInterrupt: 

In [ ]:
M = 100
D=7*7*64
h1=512
D2=10

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

tf.reset_default_graph()
# variables
X = tf.placeholder(tf.float32, shape = [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None,10])

w_conv1 = tf.get_variable('w_conv1', [5,5, 1,32], initializer = tf.contrib.layers.xavier_initializer())
w_conv2 = tf.get_variable('w_conv2', [5,5,32,64], initializer = tf.contrib.layers.xavier_initializer())

w0 = tf.get_variable('w_fc1', [7*7*64, h1], initializer = tf.contrib.layers.xavier_initializer())
b0 = tf.get_variable('b_fc1', [1,h1], initializer = tf.contrib.layers.xavier_initializer())
w1 = tf.get_variable('w_fc2', [512, 10], initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('b_fc2', [1,10], initializer = tf.contrib.layers.xavier_initializer())

# network
b=True
con1 = tf.nn.conv2d(X, w_conv1, strides=[1, 1, 1, 1], padding='SAME')
h_conv1 = tf.nn.relu(tf.layers.batch_normalization(con1, training=b))
h_pool1 = max_pool_2x2(h_conv1)

con2 = tf.nn.conv2d(h_pool1, w_conv2, strides=[1, 1, 1, 1], padding='SAME')
h_conv2 = tf.nn.relu(tf.layers.batch_normalization(con2, training=b))
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.layers.flatten(h_pool2)

h = tf.nn.relu(tf.matmul(h_pool2_flat, w0) + b0)
logits = tf.matmul(h, w1) + b1
output = tf.nn.softmax(logits)#batch*10

#evaluation
probs = tf.reduce_sum(output*Y,1)#batch*1
p=tf.reduce_mean(probs,0)#积分
#cross_ent = tf.reduce_mean(tf.reduce_sum(output*tf.log(output+1e-11),1),0)
cross_ent = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=Y)
max_p = tf.reduce_max(output,1)
ent = tf.reduce_sum(-tf.log(output+1e-11)*output,1)


correct_pred = tf.equal(tf.argmax(output,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# training
a = 0.00001
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_step = tf.train.AdamOptimizer(a).minimize(cross_ent)
saver = tf.train.Saver(max_to_keep = 200)

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    model_id=0
    #save_path = saver.restore(sess, "./baseline_point_estimation/classification_mnist60/model.ckpt")
    for i in range(90000):
        batch = mnist.train.next_batch(M)
        x_batch = np.reshape(batch[0],[M, 28, 28, 1])
        y_batch = batch[1]
        
        if i%100==0:
            b=False
            maxp_OOD = []
            ent_OOD = []
            
            test_accuracy = []
            maxp_in = []
            ent_in = []
            #accu = sess.run(accuracy,{X: np.reshape(mnist.train.images,[55000,28,28,1]), Y: mnist.train.labels})
                        
            
            for j in range(0,100):
                imgs_in = mnist.test.images[j*100:(j+1)*100]
                labels_in = mnist.test.labels[j*100:(j+1)*100]
                acc = sess.run(accuracy,{X:np.reshape(imgs_in,[100,28,28,1]),Y:labels_in})
                maxp_in1 = sess.run(max_p,{X:np.reshape(imgs_in,[100,28,28,1])})
                ent_in1 = sess.run(ent,{X:np.reshape(imgs_in,[100,28,28,1])})
                test_accuracy.append(acc)
                maxp_in.extend(maxp_in1)
                ent_in.extend(ent_in1)
            test_accuracy = np.array(test_accuracy)
            maxp_in = np.array(maxp_in)
            ent_in = np.array(ent_in)
                
            for k in range(0,92):
                imgs_OOD = safe_images[k*100:(k+1)*100]
                maxp_OOD1 = sess.run(max_p,{X:np.reshape(imgs_OOD,[100,28,28,1])})
                ent_OOD1 = sess.run(ent,{X:np.reshape(imgs_OOD,[100,28,28,1])})
                maxp_OOD.extend(maxp_OOD1)
                ent_OOD.extend(ent_OOD1)
            maxp_OOD = np.array(maxp_OOD)
            ent_OOD = np.array(ent_OOD)

            validation_error = sess.run(accuracy, {X:np.reshape(validation_x,[1000,28,28,1]), Y:validation_y})
            test_error = np.mean(np.array(test_accuracy))
            '''
            test_error = sess.run(accuracy, {X:np.reshape(mnist.test.images,[10000,28,28,1]), Y:mnist.test.labels[:5000]})
            validation_error = sess.run(accuracy, {X:np.reshape(validation_x,[1000,28,28,1]), Y:validation_y})
            maxp_OOD = sess.run(max_p,{X:np.reshape(safe_images,[9200,28,28,1])})
            maxp_in = sess.run(max_p, {X:np.reshape(mnist.test.images,[10000,28,28,1])})
            ent_OOD = sess.run(ent,{X:np.reshape(safe_images,[9200,28,28,1])})
            ent_in = sess.run(ent, {X:np.reshape(mnist.test.images,[10000,28,28,1])})
            '''
            print ("time:",i, time.time() - start_time, "test accuracy", test_error, "validation_error",validation_error)
            
            print("maxp_OOD:",np.mean(maxp_OOD>0.99),np.mean(maxp_OOD),np.std(maxp_OOD))
            print("maxp_inD:",np.mean(maxp_in>0.99),np.mean(maxp_in),np.std(maxp_in))
            print("ent_OOD:",np.mean(ent_OOD),np.std(ent_OOD), "ent_in:", np.mean(ent_in),np.std(ent_in))
            
            safe, risky  = -np.reshape(maxp_in,[10000,1]), -np.reshape(maxp_OOD,[9200,1])
            labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
            #labels[:safe.shape[0]] += 1
            labels[safe.shape[0]:] += 1
            examples = np.squeeze(np.vstack((safe, risky)))
            print('AUPR_p:', round(100*average_precision_score(labels, examples), 2))
            print('AUROC_p:', round(100*roc_auc_score(labels, examples), 2))

            safe, risky = np.reshape(ent_in,[10000,1]), np.reshape(ent_OOD,[9200,1])
            labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
            #labels[:safe.shape[0]] += 1
            labels[safe.shape[0]:] += 1
            examples = np.squeeze(np.vstack((safe, risky)))
            print('AUPR_entropy:', round(100*average_precision_score(labels, examples), 2))
            print('AUROC_entropy:', round(100*roc_auc_score(labels, examples), 2))
            print("############################################")
            
        if i < 1000:
            a = 0.005
        elif i < 5000:
            a = 0.001
        elif i < 20000:
            a = 0.0005
        elif i < 30000:
            a = 0.0001
        elif i < 40000:
            a = 0.00005
        else:
            a = 0.00001

        b = True
        sess.run(train_step,{X: x_batch, Y: y_batch})


time: 0 48.75006890296936 test accuracy 0.085099995 validation_error 0.084
maxp_OOD: 0.0 0.33909142 0.110360645
maxp_inD: 0.0 0.33035576 0.09529665
ent_OOD: 1.835069 0.20694332 ent_in: 1.8302656 0.16839695
AUPR_p: 49.45
AUROC_p: 48.82
AUPR_entropy: 55.32
AUROC_entropy: 52.75
############################################
time: 100 156.88168096542358 test accuracy 0.80100006 validation_error 0.781
maxp_OOD: 0.0 0.3367959 0.12337194
maxp_inD: 0.0 0.51756257 0.19070463
ent_OOD: 1.8813578 0.24012472 ent_in: 1.5104115 0.40854198
AUPR_p: 73.87
AUROC_p: 78.19
AUPR_entropy: 75.7
AUROC_entropy: 78.92
############################################
time: 200 262.9495806694031 test accuracy 0.88789994 validation_error 0.876
maxp_OOD: 0.00010869565217391305 0.4332685 0.1745099
maxp_inD: 0.0025 0.71335584 0.20496207
ent_OOD: 1.6310377 0.3755901 ent_in: 0.94671935 0.501152
AUPR_p: 81.28
AUROC_p: 84.08
AUPR_entropy: 83.59
AUROC_entropy: 85.62
############################################
time: 300 366.8136

time: 2500 2638.037493944168 test accuracy 0.9779999 validation_error 0.972
maxp_OOD: 0.014565217391304348 0.6362699 0.21104644
maxp_inD: 0.6528 0.96169376 0.09656354
ent_OOD: 1.0276909 0.48770037 ent_in: 0.13445717 0.2546406
AUPR_p: 92.95
AUROC_p: 94.62
AUPR_entropy: 94.02
AUROC_entropy: 95.16
############################################
time: 2600 2733.2881338596344 test accuracy 0.9795 validation_error 0.975
maxp_OOD: 0.015434782608695652 0.64079404 0.21125987
maxp_inD: 0.6638 0.9635138 0.09309549
ent_OOD: 1.0169379 0.48917127 ent_in: 0.12911506 0.24619746
AUPR_p: 93.15
AUROC_p: 94.75
AUPR_entropy: 94.21
AUROC_entropy: 95.28
############################################
time: 2700 2838.972366809845 test accuracy 0.9788 validation_error 0.975
maxp_OOD: 0.02217391304347826 0.650698 0.21365346
maxp_inD: 0.6636 0.963687 0.09304995
ent_OOD: 0.9905969 0.49821663 ent_in: 0.12820445 0.24555328
AUPR_p: 92.64
AUROC_p: 94.18
AUPR_entropy: 93.66
AUROC_entropy: 94.7
##############################

time: 5000 5041.134567975998 test accuracy 0.9858 validation_error 0.982
maxp_OOD: 0.043152173913043476 0.68744326 0.21312468
maxp_inD: 0.8049 0.9774887 0.07577868
ent_OOD: 0.8783132 0.49553612 ent_in: 0.07614005 0.19297674
AUPR_p: 93.9
AUROC_p: 95.31
AUPR_entropy: 94.79
AUROC_entropy: 95.74
############################################
time: 5100 5126.28928899765 test accuracy 0.9869 validation_error 0.986
maxp_OOD: 0.0333695652173913 0.6761528 0.21176086
maxp_inD: 0.8018 0.9773437 0.07663284
ent_OOD: 0.9063401 0.4874123 ent_in: 0.07659854 0.19397315
AUPR_p: 94.3
AUROC_p: 95.71
AUPR_entropy: 95.22
AUROC_entropy: 96.16
############################################
time: 5200 5210.879499912262 test accuracy 0.9865 validation_error 0.983
maxp_OOD: 0.03847826086956522 0.6848094 0.21251382
maxp_inD: 0.8023 0.97770196 0.075468495
ent_OOD: 0.88198996 0.49075463 ent_in: 0.07553933 0.19008835
AUPR_p: 94.06
AUROC_p: 95.48
AUPR_entropy: 94.99
AUROC_entropy: 95.92
##################################

time: 7500 7168.421238899231 test accuracy 0.989 validation_error 0.983
maxp_OOD: 0.056956521739130433 0.7061077 0.21026224
maxp_inD: 0.8534 0.98281896 0.06618902
ent_OOD: 0.8174134 0.48341727 ent_in: 0.057093665 0.16643433
AUPR_p: 94.6
AUROC_p: 95.83
AUPR_entropy: 95.35
AUROC_entropy: 96.19
############################################
time: 7600 7252.538800954819 test accuracy 0.9891 validation_error 0.987
maxp_OOD: 0.06032608695652174 0.7120528 0.21000765
maxp_inD: 0.8507 0.9826798 0.06631857
ent_OOD: 0.80212504 0.48438844 ent_in: 0.057518408 0.16597337
AUPR_p: 94.31
AUROC_p: 95.58
AUPR_entropy: 95.1
AUROC_entropy: 95.95
############################################
time: 7700 7336.290494918823 test accuracy 0.9889 validation_error 0.986
maxp_OOD: 0.050652173913043476 0.6985986 0.20991938
maxp_inD: 0.8538 0.98284745 0.06734982
ent_OOD: 0.83767 0.48192772 ent_in: 0.05641441 0.16826938
AUPR_p: 94.85
AUROC_p: 96.15
AUPR_entropy: 95.64
AUROC_entropy: 96.51
################################

time: 10000 9275.590142011642 test accuracy 0.99 validation_error 0.989
maxp_OOD: 0.06597826086956522 0.71791464 0.20836887
maxp_inD: 0.8775 0.98523515 0.06273167
ent_OOD: 0.7802809 0.47553122 ent_in: 0.047472063 0.15294498
AUPR_p: 94.83
AUROC_p: 96.15
AUPR_entropy: 95.65
AUROC_entropy: 96.5
############################################
time: 10100 9359.195894956589 test accuracy 0.99069995 validation_error 0.988
maxp_OOD: 0.07282608695652174 0.7206332 0.20869014
maxp_inD: 0.8859 0.9861184 0.059604265
ent_OOD: 0.772274 0.4789404 ent_in: 0.045169957 0.14908533
AUPR_p: 95.04
AUROC_p: 96.17
AUPR_entropy: 95.7
AUROC_entropy: 96.48
############################################
time: 10200 9443.013597011566 test accuracy 0.9893 validation_error 0.986
maxp_OOD: 0.07782608695652174 0.7280163 0.20750201
maxp_inD: 0.8773 0.98535156 0.061680604
ent_OOD: 0.75279063 0.47522196 ent_in: 0.047469635 0.15105845
AUPR_p: 94.49
AUROC_p: 95.73
AUPR_entropy: 95.25
AUROC_entropy: 96.08
########################

...

time: 27300 23923.62815475464 test accuracy 0.9915 validation_error 0.988
maxp_OOD: 0.20728260869565218 0.8071822 0.19312337
maxp_inD: 0.956 0.9937554 0.04191688
ent_OOD: 0.5139549 0.43258294 ent_in: 0.017325152 0.094032906
AUPR_p: 95.29
AUROC_p: 96.52
AUPR_entropy: 95.81
AUROC_entropy: 96.68
############################################
time: 27400 24014.724539995193 test accuracy 0.9915 validation_error 0.99
maxp_OOD: 0.20597826086956522 0.8078045 0.19291319
maxp_inD: 0.955 0.99357283 0.04336434
ent_OOD: 0.5111474 0.43107352 ent_in: 0.017489823 0.09580237
AUPR_p: 95.21
AUROC_p: 96.53
AUPR_entropy: 95.76
AUROC_entropy: 96.7
############################################
time: 27500 24105.416804790497 test accuracy 0.9918 validation_error 0.988
maxp_OOD: 0.20902173913043479 0.8091917 0.19205183
maxp_inD: 0.9562 0.9936992 0.041898686
ent_OOD: 0.5086559 0.43026108 ent_in: 0.017485319 0.09492349
AUPR_p: 95.26
AUROC_p: 96.51
AUPR_entropy: 95.76
AUROC_entropy: 96.68
###########################

time: 29800 26053.10627579689 test accuracy 0.99189997 validation_error 0.99
maxp_OOD: 0.23 0.81873333 0.18836218
maxp_inD: 0.96 0.99401844 0.041296214
ent_OOD: 0.48068747 0.41953096 ent_in: 0.016126297 0.09153156
AUPR_p: 95.18
AUROC_p: 96.45
AUPR_entropy: 95.68
AUROC_entropy: 96.61
############################################
time: 29900 26137.53702688217 test accuracy 0.9921 validation_error 0.991
maxp_OOD: 0.2442391304347826 0.82367176 0.18824211
maxp_inD: 0.9595 0.99431443 0.039957635
ent_OOD: 0.46740445 0.41984424 ent_in: 0.015462598 0.089026175
AUPR_p: 95.05
AUROC_p: 96.3
AUPR_entropy: 95.52
AUROC_entropy: 96.45
############################################
time: 30000 26220.976888895035 test accuracy 0.9913 validation_error 0.99
maxp_OOD: 0.2382608695652174 0.82212305 0.18839177
maxp_inD: 0.9609 0.99416846 0.041071042
ent_OOD: 0.47072658 0.41953453 ent_in: 0.015540295 0.08994943
AUPR_p: 95.09
AUROC_p: 96.39
AUPR_entropy: 95.6
AUROC_entropy: 96.53
#################################

time: 32300 28163.467770814896 test accuracy 0.9917 validation_error 0.99
maxp_OOD: 0.25673913043478264 0.8283322 0.18656062
maxp_inD: 0.9641 0.9946456 0.038933996
ent_OOD: 0.45247978 0.41366696 ent_in: 0.014297711 0.08637729
AUPR_p: 95.28
AUROC_p: 96.55
AUPR_entropy: 95.73
AUROC_entropy: 96.69
############################################
time: 32400 28247.732237815857 test accuracy 0.9913 validation_error 0.987
maxp_OOD: 0.2615217391304348 0.8291848 0.1866911
maxp_inD: 0.9642 0.9945862 0.03984579
ent_OOD: 0.44919488 0.41331053 ent_in: 0.014231868 0.08689328
AUPR_p: 95.17
AUROC_p: 96.48
AUPR_entropy: 95.63
AUROC_entropy: 96.62
############################################
time: 32500 28331.089991807938 test accuracy 0.9921 validation_error 0.991
maxp_OOD: 0.2608695652173913 0.83036894 0.18615213
maxp_inD: 0.9637 0.9946624 0.038485855
ent_OOD: 0.44729003 0.41259778 ent_in: 0.014383571 0.08624205
AUPR_p: 95.17
AUROC_p: 96.41
AUPR_entropy: 95.6
AUROC_entropy: 96.56
########################

time: 34800 31900.241209745407 test accuracy 0.99160004 validation_error 0.989
maxp_OOD: 0.2708695652173913 0.8352138 0.18348812
maxp_inD: 0.9661 0.9947499 0.03981352
ent_OOD: 0.43286043 0.4061805 ent_in: 0.013493529 0.08539859
AUPR_p: 95.16
AUROC_p: 96.55
AUPR_entropy: 95.65
AUROC_entropy: 96.69
############################################
time: 34900 31989.227604866028 test accuracy 0.9914 validation_error 0.987
maxp_OOD: 0.2752173913043478 0.8379347 0.1820486
maxp_inD: 0.9663 0.99490476 0.038133167
ent_OOD: 0.42679405 0.4036153 ent_in: 0.013395718 0.084080406
AUPR_p: 95.14
AUROC_p: 96.46
AUPR_entropy: 95.58
AUROC_entropy: 96.6
############################################
time: 35000 32077.66699910164 test accuracy 0.99139994 validation_error 0.989
maxp_OOD: 0.2846739130434783 0.8416345 0.18101709
maxp_inD: 0.9664 0.99494374 0.038338795
ent_OOD: 0.41757795 0.40218738 ent_in: 0.0131883975 0.08241353
AUPR_p: 94.96
AUROC_p: 96.31
AUPR_entropy: 95.47
AUROC_entropy: 96.47
################

time: 37300 33745.297880887985 test accuracy 0.9914 validation_error 0.989
maxp_OOD: 0.2975 0.84544414 0.17926742
maxp_inD: 0.9687 0.9952948 0.036894828
ent_OOD: 0.4053989 0.3957826 ent_in: 0.0121843675 0.07968412
AUPR_p: 95.21
AUROC_p: 96.54
AUPR_entropy: 95.67
AUROC_entropy: 96.7
############################################
time: 37400 33800.27921271324 test accuracy 0.9918 validation_error 0.989
maxp_OOD: 0.3017391304347826 0.84714013 0.17926483
maxp_inD: 0.9685 0.99518263 0.037400693
ent_OOD: 0.40030533 0.39498076 ent_in: 0.012376378 0.08111134
AUPR_p: 95.05
AUROC_p: 96.38
AUPR_entropy: 95.49
AUROC_entropy: 96.53
############################################
time: 37500 33852.98026180267 test accuracy 0.9918 validation_error 0.991
maxp_OOD: 0.30184782608695654 0.846837 0.17960475
maxp_inD: 0.9684 0.9954002 0.035994876
ent_OOD: 0.4013079 0.39527348 ent_in: 0.012045671 0.07879627
AUPR_p: 95.18
AUROC_p: 96.47
AUPR_entropy: 95.62
AUROC_entropy: 96.61
####################################